#  This cell import related Libraries

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 30.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
pip install nervaluate

In [ ]:
pip install pytorch-crf

In [ ]:
# in zh-dev.conll at 7078 line character of next line is added due to it was '\u200b\u200b' (mannuly)
#removed Pharsi and multi data from both m1,m2
import pandas as pd
from collections import Counter
import sys
import json
# visualization libraries
import matplotlib.pyplot as plt
import numpy as np

# pytorch libraries
import torch # the main pytorch library
import torch.nn as nn # the sub-library containing Softmax, Module and other useful functions
import torch.optim as optim # the sub-library containing the common optimizers (SGD, Adam, etc.)

# huggingface's transformers library
#from transformers import RobertaForTokenClassification, RobertaTokenizer
# from transformers import RobertaForTokenClassification, RobertaTokenizer
from transformers import XLMRobertaForTokenClassification, XLMRobertaTokenizer

# huggingface's datasets library
from datasets import load_dataset

# the tqdm library used to show the iteration progress
import tqdm
tqdmn = tqdm.notebook.tqdm#tqdm.tqdm# tqdm.notebook.tqdm

from datasets import load_dataset
import os
from transformers import get_scheduler
from datasets import Dataset
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import copy
from torchcrf import CRF
# from nervaluate import Evaluator

In [ ]:
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import requests

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def read_format(file_path):
    sentences = []
    ner_tags = []
    current_img_ids=[]

    with open(file_path, 'r', encoding='utf-8') as file:
        current_sentence = []
        current_tags = []
        for line in file:
            line = line.strip()
            if line.startswith("Img_ID:"):
                if current_sentence:
                    sentences.append(current_sentence)
                    ner_tags.append(current_tags)
                    current_img_ids.append(line)
                    current_sentence = []
                    current_tags = []

            elif line == "<s>":
                continue
            else:
                tokens = line.split("\t")
                if len(tokens) == 2:
                    current_sentence.append(tokens[0])
                    current_tags.append(tokens[1])
                else:
                    print(f"Warning: Invalid line format in {file_path}: {line}")

        # Append the last sentence and its tags
        if current_sentence:
            sentences.append(current_sentence)
            ner_tags.append(current_tags)
            current_img_ids.append(line)

    return sentences, ner_tags, current_img_ids

# Example usage:
file_path ="/content/drive/MyDrive/model_data/train_aug.txt"#'/content/drive/MyDrive/data/mhiner_train.txt'  # Update with your file path
sentences, ner_tags, current_img_ids = read_format(file_path)
current_img_ids.insert(0,"Img_ID:\t20001")
current_img_ids=current_img_ids[:-1]

# Print the sentences and corresponding NER tags
for sentence, tags in zip(sentences, ner_tags):
    print("Sentence:", sentence)
    print("NER Tags:", tags)
    print("Ncurrent_img_ids:", current_img_ids)


Streaming output truncated to the last 5000 lines.
Sentence: ['दूसरे', 'विश्व', 'युद्ध', 'में', 'भी', 'नहीं', 'हुआ', 'था', 'अमेरिका', 'का', 'इतना', 'बुरा', 'हाल,', 'इससे', 'कैसे', 'निकलेगी', 'इकॉनमी?', '<eos>', 'एक', 'सूट', 'में', 'एक', 'व्यक्ति', 'का', 'एक', 'करीबी', 'और', 'एक', 'माइक्रोफोन', 'पकड़े', 'हुए', 'टाई']
NER Tags: ['B-NUMEX', 'I-NUMEX', 'I-NUMEX', 'O', 'O', 'O', 'O', 'O', 'B-LOCATION', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-X', 'B-X', 'B-X', 'B-X', 'B-X', 'B-X', 'B-X', 'B-X', 'B-X', 'B-X', 'B-X', 'B-X', 'B-X', 'B-X', 'B-X']
Ncurrent_img_ids: ['Img_ID:\t20001', 'Img_ID:\t20002', 'Img_ID:\t20003', 'Img_ID:\t20004', 'Img_ID:\t20005', 'Img_ID:\t20006', 'Img_ID:\t20007', 'Img_ID:\t20008', 'Img_ID:\t20009', 'Img_ID:\t20010', 'Img_ID:\t20011', 'Img_ID:\t20012', 'Img_ID:\t20013', 'Img_ID:\t20014', 'Img_ID:\t20015', 'Img_ID:\t20016', 'Img_ID:\t20017', 'Img_ID:\t20018', 'Img_ID:\t20019', 'Img_ID:\t20020', 'Img_ID:\t20021', 'Img_ID:\t20022', 'Img_ID:\t20023', 'Img_ID:\t20024', 'Im

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
current_img_ids[-5:]

['Img_ID:\t51883',
 'Img_ID:\t51884',
 'Img_ID:\t51885',
 'Img_ID:\t51886',
 'Img_ID:\t51887']

In [ ]:
print(current_img_ids[-5:-1])
print(sentences[-5:-1])

['Img_ID:\t51883', 'Img_ID:\t51884', 'Img_ID:\t51885', 'Img_ID:\t51886']
[['Bulk', 'Deals', ':', 'Pastel', 'ने', 'Bharti', 'Airtel', 'में', 'बेचे', '5,849', 'करोड़', 'रुपये', 'के', 'शेयर,', 'M&M', 'के', 'प्रमोटर', 'ने', 'भी', 'की', 'बिकवाली', '<eos>', 'एक', 'सूट', 'में', 'एक', 'आदमी', 'उस', 'पर', 'एक', 'चार्ट', 'के', 'साथ', 'अपने', 'हाथों', 'को', 'पकड़े', 'हुए'], ['Sidhu', 'Moosewala', 'की', 'मां', 'की', 'IVF', 'पर', 'सवाल', 'किए', 'जा', 'रहे', 'हैं', 'खड़े,', 'उम्र', 'को', 'लेकर', 'हुआ', 'बवाल', '<eos>', 'पगड़ी', 'में', 'अराफेड', 'आदमी', 'एक', 'बच्चे', 'और', 'एक', 'महिला', 'को', 'गुलाबी', 'पोशाक', 'में', 'पकड़े', 'हुए'], ['Leap', 'Day', '2024:', 'पहले', '30', 'फरवरी', 'की', 'तारीख', 'भी', 'आती', 'थी,', 'फिर', 'हटा', 'दी', 'गई,', 'जानिए', 'क्या', 'है', 'इसका', 'रहस्य', '<eos>', 'उस', 'पर', 'दिनांक', '30', 'के', 'साथ', 'एक', 'कैलेंडर', 'का', 'एक', 'क्लोज', 'अप'], ['Aaj', 'Ki', 'Taza', 'Khabar:', 'पढ़ें', '29', 'मार्च', '2024', 'की', 'सुबह', 'की', 'टॉप', 'खबरें', 'और', 'अन्य', 'समाचार', '

In [ ]:
print(current_img_ids[:20])
for i in sentences[:20]:
    print(i)
    print()

['Img_ID:\t20001', 'Img_ID:\t20002', 'Img_ID:\t20003', 'Img_ID:\t20004', 'Img_ID:\t20005', 'Img_ID:\t20006', 'Img_ID:\t20007', 'Img_ID:\t20008', 'Img_ID:\t20009', 'Img_ID:\t20010', 'Img_ID:\t20011', 'Img_ID:\t20012', 'Img_ID:\t20013', 'Img_ID:\t20014', 'Img_ID:\t20015', 'Img_ID:\t20016', 'Img_ID:\t20017', 'Img_ID:\t20018', 'Img_ID:\t20019', 'Img_ID:\t20020']
['दूसरे', 'विश्व', 'युद्ध', 'में', 'भी', 'नहीं', 'हुआ', 'था', 'अमेरिका', 'का', 'इतना', 'बुरा', 'हाल,', 'इससे', 'कैसे', 'निकलेगी', 'इकॉनमी?', '<eos>', 'एक', 'सूट', 'में', 'एक', 'व्यक्ति', 'का', 'एक', 'करीबी', 'और', 'एक', 'माइक्रोफोन', 'पकड़े', 'हुए', 'टाई']

['दुनिया', 'में', 'LIC', 'ने', 'मारी', 'बाजी,', 'चीन', 'के', 'दबदबे', 'के', 'सामने', 'भारतीय', 'ब्रांड', 'ने', 'किया', 'टॉप', '<eos>', 'उस', 'पर', 'एक', 'लोगो', 'के', 'साथ', 'एक', 'व्यवसाय', 'कार्ड', 'के', 'करीब']

['UltraTech', 'Cement', 'को', 'छत्तीसगढ़', 'सरकार', 'ने', 'जारी', 'किया', 'डिमांड', 'ऑर्डर,', '₹21', 'करोड़', 'भरने', 'का', 'फरमान', '<eos>', 'अराफ', 'टेक', 'सीमेंट',

In [ ]:
#AerospaceManufacturer,CarManufacturer are treated as corp
labels_list = ["O", "B-FESTIVAL", "B-GAME", "B-LANGUAGE", "B-LITERATURE", "B-LOCATION", "B-MISC", "B-NUMEX", "B-ORGANIZATION", "B-PERSON", "B-RELIGION", "B-TIMEX", "I-FESTIVAL", "I-GAME", "I-LANGUAGE", "I-LITERATURE", "I-LOCATION", "I-MISC", "I-NUMEX", "I-ORGANIZATION", "I-PERSON", "I-RELIGION", "I-TIMEX","B-X"]
labels_list_1 = ["O", "FESTIVAL", "GAME", "LANGUAGE", "LITERATURE", "LOCATION", "MISC", "NUMEX", "ORGANIZATION", "PERSON", "RELIGION", "TIMEX"]

label2id = {}
id2label ={}
for i in range(len(labels_list)):
    label2id[labels_list[i]] = i
    id2label[i] = labels_list[i]


In [ ]:
def convert_labels2id_list(list1):
  converted = [label2id[tag] for tag in list1 ]
  return converted

In [ ]:
ner_tags = [convert_labels2id_list(list1) for list1 in ner_tags]
ner_tags[:5]

[[7,
  18,
  18,
  0,
  0,
  0,
  0,
  0,
  5,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  23,
  23,
  23,
  23,
  23,
  23,
  23,
  23,
  23,
  23,
  23,
  23,
  23,
  23,
  23],
 [0,
  0,
  8,
  0,
  0,
  18,
  0,
  5,
  0,
  0,
  0,
  0,
  6,
  19,
  0,
  0,
  23,
  23,
  23,
  23,
  23,
  23,
  23,
  23,
  23,
  23,
  23,
  23],
 [8,
  19,
  0,
  8,
  19,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  18,
  0,
  0,
  23,
  23,
  23,
  23,
  23,
  23,
  23,
  23,
  23,
  23,
  23,
  23,
  23],
 [8,
  19,
  19,
  19,
  0,
  0,
  7,
  18,
  0,
  0,
  0,
  0,
  0,
  0,
  23,
  23,
  23,
  23,
  23,
  23,
  23,
  23,
  23,
  23,
  23,
  23,
  23,
  23],
 [6,
  18,
  0,
  5,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  20,
  7,
  23,
  23,
  23,
  23,
  23,
  23,
  23,
  23,
  23,
  23,
  23,
  23,
  23]]

In [ ]:
c=0
for sentence, tags in zip(sentences, ner_tags):
    c+=1
    if c==20:
      break
    print("Sentence:", sentence)
    print("NER Tags:", tags)


Sentence: ['दूसरे', 'विश्व', 'युद्ध', 'में', 'भी', 'नहीं', 'हुआ', 'था', 'अमेरिका', 'का', 'इतना', 'बुरा', 'हाल,', 'इससे', 'कैसे', 'निकलेगी', 'इकॉनमी?', '<eos>', 'एक', 'सूट', 'में', 'एक', 'व्यक्ति', 'का', 'एक', 'करीबी', 'और', 'एक', 'माइक्रोफोन', 'पकड़े', 'हुए', 'टाई']
NER Tags: [7, 18, 18, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23]
Sentence: ['दुनिया', 'में', 'LIC', 'ने', 'मारी', 'बाजी,', 'चीन', 'के', 'दबदबे', 'के', 'सामने', 'भारतीय', 'ब्रांड', 'ने', 'किया', 'टॉप', '<eos>', 'उस', 'पर', 'एक', 'लोगो', 'के', 'साथ', 'एक', 'व्यवसाय', 'कार्ड', 'के', 'करीब']
NER Tags: [0, 0, 8, 0, 0, 18, 0, 5, 0, 0, 0, 0, 6, 19, 0, 0, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23]
Sentence: ['UltraTech', 'Cement', 'को', 'छत्तीसगढ़', 'सरकार', 'ने', 'जारी', 'किया', 'डिमांड', 'ऑर्डर,', '₹21', 'करोड़', 'भरने', 'का', 'फरमान', '<eos>', 'अराफ', 'टेक', 'सीमेंट', 'साइन', 'एक', 'आदमी', 'द्वारा', 'साफ', 'किया', 'जा', 'रहा', 'है']
NER Tags: [8, 19, 0, 8, 19, 0, 0,

In [ ]:
label2id

{'O': 0,
 'B-FESTIVAL': 1,
 'B-GAME': 2,
 'B-LANGUAGE': 3,
 'B-LITERATURE': 4,
 'B-LOCATION': 5,
 'B-MISC': 6,
 'B-NUMEX': 7,
 'B-ORGANIZATION': 8,
 'B-PERSON': 9,
 'B-RELIGION': 10,
 'B-TIMEX': 11,
 'I-FESTIVAL': 12,
 'I-GAME': 13,
 'I-LANGUAGE': 14,
 'I-LITERATURE': 15,
 'I-LOCATION': 16,
 'I-MISC': 17,
 'I-NUMEX': 18,
 'I-ORGANIZATION': 19,
 'I-PERSON': 20,
 'I-RELIGION': 21,
 'I-TIMEX': 22,
 'B-X': 23}

In [ ]:
from sklearn.model_selection import train_test_split

def split_data(sentences, ner_tags, test_size=0.2, random_state=None):
    # Split the data into train and validation sets
    train_sentences, valid_sentences, train_ner_tags, valid_ner_tags, train_current_img_ids, valid_current_img_ids = train_test_split(
        sentences, ner_tags,current_img_ids, test_size=test_size, random_state=random_state)

    return train_sentences, valid_sentences, train_ner_tags, valid_ner_tags, train_current_img_ids, valid_current_img_ids

# Example usage:
# sentences = [...]  # Your list of sentences
# ner_tags = [...]   # Your list of NER tags

train_sentences, valid_sentences, train_ner_tags, valid_ner_tags, train_current_img_ids, valid_current_img_ids = split_data(sentences, ner_tags, test_size=0.2, random_state=42)

# test_path = "/home/naive123/nlp/Sumit/mner/data/testing.txt"#'/content/drive/MyDrive/data/testing.txt'
# test_sentences, test_ner_tags = read_format(test_path)
# test_ner_tags = [convert_labels2id_list(list1) for list1 in test_ner_tags]
# test_ner_tags[:5]
# # Print the sentences and corresponding NER tags
# for sentence, tags in zip(test_sentences, test_ner_tags):
#     print("Sentence:", sentence)
#     print("NER Tags:", tags)

print("Number of training sentences:", len(train_sentences))
print("Number of validation sentences:", len(valid_sentences))

Number of training sentences: 4308
Number of validation sentences: 1078


In [ ]:
print("Number of validation sentences:", len(train_current_img_ids))

Number of validation sentences: 4308


In [ ]:
print(len(train_ner_tags))

4308


In [ ]:

train_pdf = pd.DataFrame({"sentences":train_sentences,"labels":train_ner_tags,"id":train_current_img_ids})
dev_pdf = pd.DataFrame({"sentences":valid_sentences,"labels":valid_ner_tags,"id":valid_current_img_ids})
# test_pdf =pd.DataFrame({"sentences":test_sentences,"labels":test_ner_tags,"id":0})




################################################

In [ ]:
train_pdf

,sentences,labels,id
0,"[सरकार, फसल, एमएसपी, पर, खरीदे, :, सतबीर, भाणा...","[8, 18, 8, 0, 0, 0, 9, 20, 23, 23, 23, 23, 23,...",Img_ID:\t51031
1,"[कांग्रेस, में, विलय, के, बाद, राहुल-प्रियंका,...","[8, 0, 0, 0, 0, 9, 18, 9, 0, 0, 0, 0, 9, 23, 2...",Img_ID:\t20052
2,"[पूर्व, मंत्री, मनीष, ग्रोवर, के, भाई, के, निध...","[7, 18, 9, 20, 0, 7, 0, 0, 0, 8, 0, 0, 0, 23, ...",Img_ID:\t50433
3,"[निफ्टी, और, बैंक, निफ्टी, के, लिए, कैसे, हैं,...","[5, 0, 8, 19, 0, 0, 0, 0, 0, 0, 0, 11, 0, 7, 1...",Img_ID:\t30710
4,"[क्या, डिप्रेशन, में, ज्यादा, खाने, लगते, हैं,...","[0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 18, 0, 0, 6, 0,...",Img_ID:\t20746
...,...,...,...
4303,"[अरविंद, केजरीवाल, पर, IIT, खड़गपुर, में, रेप,...","[9, 20, 0, 11, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Img_ID:\t31436
4304,"[‘केंद्र,, प्रदेश, सरकार, किसान, विरोधी’, <eos...","[11, 8, 18, 8, 19, 23, 23, 23, 23, 23, 23, 23,...",Img_ID:\t50274
4305,"[अमेरिका, के, H-1B, वीजा, के, रजिस्ट्रेशन, होन...","[5, 22, 7, 18, 7, 18, 0, 0, 0, 0, 0, 0, 0, 23,...",Img_ID:\t51693
4306,"[समुद्र, में, जब, डकैतों, का, होता, है, हमला,,...","[5, 0, 0, 6, 0, 0, 0, 0, 0, 0, 7, 16, 0, 8, 0,...",Img_ID:\t51728


In [ ]:
dev_pdf

,sentences,labels,id
0,"[15, दिन,, 3, राज्य..., तीन, बार, बिक्री,, बीज...","[11, 22, 0, 7, 18, 18, 18, 18, 7, 18, 0, 0, 8,...",Img_ID:\t51589
1,"[प्यास, लगी, तो, कुआं, खोदेंगे,, यह, नजरिया, ब...","[7, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 8, 19, 19, 0...",Img_ID:\t30919
2,"[आज, ABB, Power, और, Vedanta, सहित, इन, शेयरों...","[11, 8, 19, 16, 5, 0, 7, 0, 0, 0, 0, 0, 0, 0, ...",Img_ID:\t31678
3,"[श्रीलंका, को, कच्चातिवु, देने, पर, हो, रही, ह...","[5, 0, 7, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 23...",Img_ID:\t20085
4,"[भारतीय, चालक, दल, को, हमने, नहीं, पकड़ा..., ई...","[8, 19, 19, 0, 0, 0, 0, 0, 0, 7, 5, 0, 0, 7, 6...",Img_ID:\t51153
...,...,...,...
1073,"[मेष-कुंभ, राशि, वालों, को, मिलेगी, खुशखबरी,, ...","[11, 18, 11, 18, 14, 0, 0, 0, 0, 0, 7, 23, 23,...",Img_ID:\t50642
1074,"[प्रियंका, गांधी, की, सुरक्षा, को, जा, रही, फॉ...","[9, 20, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 7, 18...",Img_ID:\t51136
1075,"[नर्सरी, की, रेस, हुई, और, मुश्किल..., 25%, फ्...","[5, 0, 0, 0, 0, 0, 0, 0, 7, 7, 18, 6, 18, 0, 0...",Img_ID:\t20622
1076,"[BSP, की, कैंडिडेट, लिस्ट, में, 7, मुस्लिम, तो...","[8, 0, 5, 16, 0, 7, 10, 0, 7, 18, 18, 0, 9, 0,...",Img_ID:\t20567


In [ ]:
# test_pdf

# Select and load tokenizer and model

# "Course grained checkpoints"



In [ ]:
#run this cell only for training
#set Hyperparameter
# Set Hyperparameters
img_d= 768
num_labels =24
model_max_length = 92
lr=1e-05
n_epochs = 10
batch_size= 32
min_batch_size=8
batch_size_c = batch_size / min_batch_size
early_stoping_patience = 3 # if valid loss will increase 3 consecutive times with a margin then training will stop.
tokenizer_path = "FacebookAI/xlm-roberta-large"
initial_model = "FacebookAI/xlm-roberta-large"#FacebookAI/xlm-roberta-large

In [ ]:
#load tokenizer and model

tokenizer = XLMRobertaTokenizer.from_pretrained(tokenizer_path)
tokenizer.model_max_length = model_max_length
print(tokenizer.model_max_length )

# initialize the model and provide the 'num_labels' used to create the classification layer
model = XLMRobertaForTokenClassification.from_pretrained(initial_model,num_labels=num_labels)    #, num_labels=num_labels)
# assign the 'id2label' and 'label2id' model configs
model.config.id2label = id2label
model.config.label2id = label2id
print("model loded")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

92


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model loded


# Prepare data for model
#  If a tokanized sentence length is > 92 then we will cut the tokanized #sentence at 92 ( max length = 92)
# Word id created
# Labels allignment (expandation) according to tokanized sentences.

In [ ]:
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
# def image_feature(image_id)
#     # img_id = '/home/naive123/nlp/Sumit/mner/downloaded_images/20002.jpg'
#     image = Image.open(url)


#     inputs = processor(images=image, return_tensors="pt")

#     outputs = model(**inputs)
#     last_hidden_states = outputs.last_hidden_state

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [ ]:
!unzip /content/drive/MyDrive/model_data/downloaded_img.zip -d /content/drive/MyDrive/model_data/

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/model_data/downloaded_images/30475.jpg  
  inflating: /content/drive/MyDrive/model_data/__MACOSX/downloaded_images/._30475.jpg  
  inflating: /content/drive/MyDrive/model_data/downloaded_images/20666.jpg  
  inflating: /content/drive/MyDrive/model_data/__MACOSX/downloaded_images/._20666.jpg  
  inflating: /content/drive/MyDrive/model_data/downloaded_images/21578.jpg  
  inflating: /content/drive/MyDrive/model_data/__MACOSX/downloaded_images/._21578.jpg  
  inflating: /content/drive/MyDrive/model_data/downloaded_images/50381.jpg  
  inflating: /content/drive/MyDrive/model_data/__MACOSX/downloaded_images/._50381.jpg  
  inflating: /content/drive/MyDrive/model_data/downloaded_images/20100.jpg  
  inflating: /content/drive/MyDrive/model_data/__MACOSX/downloaded_images/._20100.jpg  
  inflating: /content/drive/MyDrive/model_data/downloaded_images/30313.jpg  
  inflating: /content/drive/MyDrive/model_data/

In [ ]:
#prepare data for model
#If a tokanized sentence length is > 92 then we will cut the tokanized sentenc at 92 ( max length = 92)
word_ids_of_sentences = []



def add_encodings(example):
    global word_ids_of_sentences
    # print(example)


    # For allign_labels with tokenized sentence

    word_id_value=0
    word_ids_for_curr_sentence = []


    for word in example['sentences']:
        print(word)

        tokenized_word = tokenizer.convert_ids_to_tokens(tokenizer(word,is_split_into_words=False)["input_ids"],skip_special_tokens=False)[1:-1]
        # print(tokenized_word)
        if(len(tokenized_word) == 0):
            print(example)
            print(tokenized_word)
            sss

        word_ids_for_curr_sentence += ([word_id_value]*len(tokenized_word))
        if len(word_ids_for_curr_sentence)>=90:
            word_ids_for_curr_sentence = word_ids_for_curr_sentence[:90]
            break

        word_id_value+=1


    word_ids_of_sentences.append(word_ids_for_curr_sentence)
    #for printing special tokens
    #print(tokenizer.convert_ids_to_tokens(tokenizer("sumit",is_split_into_words=False)["input_ids"],skip_special_tokens=False))


    encodings = tokenizer(example['sentences'], truncation=True, padding='max_length', is_split_into_words=True)



    if sum(encodings["attention_mask"]) != len(word_ids_for_curr_sentence)+2:
        if (len(word_ids_for_curr_sentence)+2) <=90:

            print(len(word_ids_for_curr_sentence),sum(encodings["attention_mask"]))
            print( tokenizer.convert_ids_to_tokens(tokenizer(example['sentences'], truncation=True, is_split_into_words=True)["input_ids"],skip_special_tokens=False)[1:-1])
            print(word_ids_for_curr_sentence)
            sss

    ex_lab=example['labels']

    extended_labels=[]

    for  w_id in word_ids_for_curr_sentence:
        extended_labels.append(ex_lab[w_id])
    extended_labels.insert(0,0)         #for special token generated by tokenizer
    extended_labels.append(0)           #for special token generated by tokenizer
    labels_ = extended_labels+ [0] * (tokenizer.model_max_length - len(extended_labels))  #padding
    if (len(labels_)>92):
        labels_=labels_[:92]

    image = Image.open("/content/drive/MyDrive/model_data/downloaded_images/"+example["id"].split("\t")[1] +".jpg")
    if image.mode != "RGB":
        image = image.convert("RGB")

    image_pt = processor(images=image, return_tensors="pt")

    outputs = model(**image_pt)
    last_hidden_states = outputs.last_hidden_state
    # encodings.append(last_hidden_states[0][0].cpu().detach().numpy())
    # return the encodings and the extended ner_tags
    print("complete")
    # print(last_hidden_states[0][0].shape)
    return { **encodings, 'labels': labels_,"feat": last_hidden_states[0][0].cpu().detach().numpy()}

In [ ]:
valid_dataset = Dataset.from_pandas(dev_pdf.iloc[:5])
#valid_dataset=valid_dataset.map(add_encodings)

In [ ]:
valid_dataset[:3]

{'sentences': [['15',
   'दिन,',
   '3',
   'राज्य...',
   'तीन',
   'बार',
   'बिक्री,',
   'बीजेपी',
   'चीफ',
   'जेपी',
   'नड्डा',
   'की',
   'पत्नी',
   'की',
   'फॉर्च्यूनर',
   'मिलने',
   'की',
   'इनसाइड',
   'स्टोरी',
   '<eos>',
   'एक',
   'सफेद',
   'एसयूवी',
   'के',
   'बगल',
   'में',
   'एक',
   'बनियान',
   'और',
   'चश्मा',
   'में',
   'एक',
   'आदमी'],
  ['प्यास',
   'लगी',
   'तो',
   'कुआं',
   'खोदेंगे,',
   'यह',
   'नजरिया',
   'बदलना',
   'होगा,',
   'बॉम्बे',
   'हाई',
   'कोर्ट',
   'ने',
   'एकनाथ',
   'शिंदे',
   'सरकार',
   'से',
   'क्यों',
   'कहा',
   'ऐसा?',
   '<eos>',
   'दाढ़ी',
   'और',
   'चश्मा',
   'वाला',
   'एक',
   'आदमी',
   'एक',
   'इमारत',
   'के',
   'सामने',
   'खड़ा',
   'है'],
  ['आज',
   'ABB',
   'Power',
   'और',
   'Vedanta',
   'सहित',
   'इन',
   'शेयरों',
   'में',
   'कमाई',
   'के',
   'मौके,',
   'क्\u200dया',
   'लगाएंगे',
   'दांव?',
   '<eos>',
   'एक',
   'सूट',
   'में',
   'एक',
   'आदमी',
   'की',
   'छवि',
   'एक

In [ ]:
import os

file_path = '/content/drive/MyDrive/model_data/downloaded_images/51589.jpg'

if os.path.exists(file_path):
    print("The file exists.")
else:
    print("The file does not exist or the path is incorrect.")


The file exists.


In [ ]:
valid_data = valid_dataset.map(add_encodings)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

15
दिन,
3
राज्य...
तीन
बार
बिक्री,
बीजेपी
चीफ
जेपी
नड्डा
की
पत्नी
की
फॉर्च्यूनर
मिलने
की
इनसाइड
स्टोरी
<eos>
एक
सफेद
एसयूवी
के
बगल
में
एक
बनियान
और
चश्मा
में
एक
आदमी
complete
प्यास
लगी
तो
कुआं
खोदेंगे,
यह
नजरिया
बदलना
होगा,
बॉम्बे
हाई
कोर्ट
ने
एकनाथ
शिंदे
सरकार
से
क्यों
कहा
ऐसा?
<eos>
दाढ़ी
और
चश्मा
वाला
एक
आदमी
एक
इमारत
के
सामने
खड़ा
है
complete
आज
ABB
Power
और
Vedanta
सहित
इन
शेयरों
में
कमाई
के
मौके,
क्‍या
लगाएंगे
दांव?
<eos>
एक
सूट
में
एक
आदमी
की
छवि
एक
स्क्रीन
पर
एक
उंगली
को
छू
रही
है
complete
श्रीलंका
को
कच्चातिवु
देने
पर
हो
रही
है
सिर्फ
राजनीति,
जानिए
क्या
है
असलियत
<eos>
एक
नक्शे
के
पास
पानी
के
एक
शरीर
पर
एक
नाव
के
पास
complete
भारतीय
चालक
दल
को
हमने
नहीं
पकड़ा...
ईरान
का
जब्‍त
'इजरायली'
जहाज
पर
बड़ा
बयान,
बताया
कहां
फंसा
है
पेच
<eos>
एक
सूट
और
चश्मा
में
एक
आदमी
एक
भारतीय
रिपोर्टर
से
बात
कर
रहा
है
complete


In [ ]:
valid_data

Dataset({
    features: ['sentences', 'labels', 'id', 'input_ids', 'attention_mask', 'feat'],
    num_rows: 5
})

In [ ]:
train_dataset =  Dataset.from_pandas(train_pdf)
train_labels = train_dataset['labels']
valid_dataset = Dataset.from_pandas(dev_pdf)




valid_labels = valid_dataset['labels']
valid_true_flat_labels = [item for sublist in valid_labels for item in sublist]



word_ids_of_test_sentences = []




print("encodinmmg start")
train_dataset = train_dataset.map(add_encodings)
word_ids_of_train_sentences = word_ids_of_sentences
word_ids_of_sentences=[]

valid_dataset=valid_dataset.map(add_encodings)
word_ids_of_valid_sentences=word_ids_of_sentences
word_ids_of_sentences=[]




print("encodinmmg complete")





train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
valid_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

train_data = torch.utils.data.DataLoader(train_dataset, batch_size=min_batch_size)
valid_data = torch.utils.data.DataLoader(valid_dataset, batch_size=min_batch_size)


NameError: name 'Dataset' is not defined

In [ ]:
# Number of batches for each data
print(len(train_data))
print(train_data)
print(len(valid_data))
print(valid_data)


NameError: name 'train_data' is not defined

# For testing only run next cell

In [ ]:
test_dataset =  Dataset.from_pandas(test_pdf)
test_labels = test_dataset['labels']
test_true_flat_labels = [item for sublist in test_labels for item in sublist]



# bn_test_dataset = Dataset.from_pandas(bn_test_pdf)
# bn_test_labels = bn_test_dataset['labels']
# bn_test_true_flat_labels = [item for sublist in bn_test_labels for item in sublist]



word_ids_of_sentences=[]





print("encodinmmg start")
test_dataset = test_dataset.map(add_encodings)
word_ids_of_test_sentences = word_ids_of_sentences
word_ids_of_sentences=[]

print("encodinmmg complete")





test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

test_data = torch.utils.data.DataLoader(test_dataset, batch_size=min_batch_size)
len(test_data)




NameError: name 'test_pdf' is not defined

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:

linear_layer = nn.Linear(1024 + img_d, num_labels)
drop=nn.Dropout(0.1).to(device)

crf_model = CRF(num_labels).to(device)

l_lr = 1e-5
crf_lr = 1e-5

# optimizer = optim.AdamW([ { 'params':model.parameters(), 'lr':lr},{'params':weight_layer, 'lr':crf_lr},{'params':bias_layer, 'lr':crf_lr} ])
optimizer = optim.AdamW([ { 'params':model.parameters(), 'lr':lr},{'params':linear_layer.parameters(), 'lr':l_lr},{'params':crf_model.parameters(), 'lr':crf_lr}])

num_training_steps = (n_epochs * len(train_data))/16
# lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=100, num_training_steps=num_training_steps)

In [ ]:
train_dataset

Dataset({
    features: ['sentences', 'labels', 'id', 'input_ids', 'attention_mask', 'feat'],
    num_rows: 4356
})

# Run next 4 cells only for training or skip

In [ ]:
# Check if 'feat' key is present in the batch dictionary
if 'feat' in batch:
    # 'feat' key is present
    image_id = batch.pop('feat').to(device)
    # Rest of your code here...
else:
    # 'feat' key is not present
    # Handle this case appropriately, such as raising an error or skipping the current batch
    print("Error: 'feat' key not found in the batch dictionary.")


In [ ]:
#run this cell only for training

num_training_steps = n_epochs * len(train_data)#/batch_size

optimizer = optim.AdamW(params=model.parameters(), lr=lr)
lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=5, num_training_steps=num_training_steps)

print("training start")


previous_total_valid_loss = sys.float_info.max
valid_min_loss = previous_total_valid_loss
early_stoping_flag=0
train_loss_epoch=[]
valid_loss_epoch=[]
directory = "./models_img/mner_"+str(lr)+"_"+str(batch_size)+"_"+str(n_epochs)+"_/"
# iterate through the data 'n_epochs' times
for epoch in tqdmn(range(n_epochs)):
    # set the model in 'train' mode and send it to the device
    model.train().to(device)
    print(epoch)
    print(directory)

    train_loss = []
    current_loss = 0
    total_loss=0
    # iterate through each batch of t    he train data
    for i, batch in enumerate(tqdmn(train_data)):

        image_id = batch.pop('feat').to(device)
        # move the batch tensors to the same device as the
        batch = { k: v.to(device) for k, v in batch.items() }
        # send 'input_ids', 'attention_mask' and 'labels' to the model
        # print(len(batch["labels"][0]))
        # print(len(batch["input_ids"][0]))
        # print(batch)

        outputs = model(**batch)
        image_id=image_id.repeat(batch_size,1)
        print(image_id.shape)
        print(outputs.last_hidden_state.shape)
        print("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
        augmented_layer = torch.cat((outputs.last_hidden_state, image_id), 1)
        #print(outputs[1])


        loss = outputs[0]
        #print("loss",loss)
        loss.backward()


        current_loss += loss.item()

        if i % 4 == 0 and i > 0: #if i >= 0:  # #           #if i % 8 == 0 and i > 0:
            # update the model using the optimizer
            optimizer.step()
            lr_scheduler.step()
            # once we update the model we set the gradients to zero
            optimizer.zero_grad()
            # store the loss value for visualization
            train_loss.append(current_loss/batch_size)
            total_loss=total_loss + current_loss/batch_size
            current_loss = 0
    train_loss_epoch.append(total_loss)
    # update the model one last time for this epoch
    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    model.save_pretrained(directory+"best_e_"+str(epoch))

    print(directory + ' files saved')
    print('total_loss',total_loss)

    #validation

    model.eval().to(device)
    valid_loss = []
    valid_current_loss = 0
    valid_total_loss=0

    # iterate through each batch of t    he train data
    for i, batch in enumerate(tqdmn(valid_data)):

        # move the batch tensors to the same device as the
        batch = { k: v.to(device) for k, v in batch.items() }
        # send 'input_ids', 'attention_mask' and 'labels' to the model
        outputs = model(**batch)
        # the outputs are of shape (loss, logits)
        #print(outputs[1])


        loss = outputs[0]
        #print("loss",loss)


        valid_current_loss += loss.item()
        #print(valid_current_loss)

        if i % batch_size_c == 0 and i > 0: #depend on batch size and GPU

            valid_loss.append(valid_current_loss/batch_size)
            valid_total_loss=valid_total_loss + valid_current_loss/batch_size
            valid_current_loss = 0
            #print("valid_total_loss ",valid_total_loss)

    print("valid_total_loss ",valid_total_loss)
    valid_loss_epoch.append(valid_total_loss)
    if valid_total_loss < valid_min_loss:
        valid_min_loss = valid_total_loss
        #model.save_pretrained(directory+"best_e_"+str(epoch))


    if previous_total_valid_loss < valid_total_loss:
        early_stoping_flag += 1
    else:
        early_stoping_flag=0
    previous_total_valid_loss = valid_total_loss


    if early_stoping_flag == early_stoping_patience:
        print("early_stoping occure")
        break



print('training complete')




training start


  0%|          | 0/10 [00:00<?, ?it/s]

0
./models_img/mner_1e-05_32_10_/


  0%|          | 0/545 [00:00<?, ?it/s]

KeyError: 'feat'

In [ ]:
"""
Visualize the training loss through the iterations.
"""

fig, ax = plt.subplots(figsize=(10, 4))
# visualize the loss values
ax.plot(train_loss)
# set the labels
ax.set_ylabel('Loss')
ax.set_xlabel('Iterations (64 examples)')
fig.tight_layout()
plt.show()

In [ ]:
"""
Visualize the training and valid loss through the epochs.
"""
print(train_loss_epoch)
print(valid_loss_epoch)
#loss normalization (loss oer batch)
loss1=[i/len(train_data) for i in train_loss_epoch]
loss2=[i/len(valid_data) for i in valid_loss_epoch]
epoch=[i for i in range(len(loss1))]


plt.plot(epoch,loss1,label=['Train'])
plt.plot(epoch,loss2,label=['Valid'])
# plt.plot(loss2)

# Adding Title to the Plot
plt.title("Training and valid loss")

# Setting the X and Y labels
plt.xlabel('epoch',fontweight='bold')
plt.ylabel('loss',fontweight='bold')
plt.legend()

plt.show()
epoch

In [ ]:
best_model_path

# Prediction on M2

In [ ]:
#best_model_path="/home/naive238/nlp/Sumit/multiconer2/ce/model/xlm_r_m2_fg_checkpoint_5e-06_64_10_epoch10_finetune_it/x_1e-06_64_20_/best_e_2"
#best_model_path="/home/naive238/nlp/Sumit/multiconer2/ce/model/xlm_r_m2_fg_checkpoint_1e-06_64_10_epoch10_finetune_hi/x_1e-06_64_15/best_e_14"
#best_model_path = "/home/naive238/nlp/Sumit/multiconer2/ce/model/xlm_r_m2_fg_checkpoint_5e-06_64_10_epoch10_finetune_de/x_1e-06_64_20_/best_e_19"
#best_model_path="/home/naive238/nlp/Sumit/multiconer2/ce/model/xlm_r_m2_fg_checkpoint_5e-06_64_10_epoch10_finetune_en/x_1e-06_64_20_/best_e_13"
#best_model_path"/home/naive238/nlp/Sumit/multiconer2/ce/model/xlm_r_m2_fg_checkpoint_5e-06_64_10_epoch10_finetune_it/x_1e-06_64_20_/best_e_2"
#best_model_path="/home/naive238/nlp/Sumit/multiconer2/ce/model/xlm_r_m2_fg_checkpoint_5e-06_64_10_epoch10_finetune_fr/x_1e-06_64_20_/best_e_4"
#best_model_path="/home/naive238/nlp/Sumit/multiconer2/ce/model/xlm_r_m2_fg_checkpoint_5e-06_64_10_epoch10_finetune_uk/x_1e-06_64_20_/best_e_15"
#best_model_path="/home/naive238/nlp/Sumit/multiconer2/ce/model/xlm_r_m2_fg_checkpoint_5e-06_64_10_epoch10_finetune_sv/x_1e-06_64_20_/best_e_19"

best_model_path="./models/mner_5e-06_32_10_/best_e_9"
num_labels = 24
model_max_length = 92
# batch_size= 64
# min_batch_size=16
# batch_size_c = batch_size / min_batch_size



In [ ]:
# Prediction
def  prediction(best_model_path, eval_data):

    confusion = torch.zeros(num_labels, num_labels)
    predicted=[]
    tokanised_exam=[]
    true_labels=[]

    confusion = torch.zeros(num_labels, num_labels)
    predicted_for_classification_report=[]

    ##################################################
    #load best model


    print(best_model_path)

    model = XLMRobertaForTokenClassification.from_pretrained(best_model_path)



    model = model.eval().to(device)
    print("model device")
    # batch the train data so that each batch contains 4 examples (using 'batch_size')
    #test_data = torch.utils.data.DataLoader(test_dataset, batch_size=16)

    eval_data = eval_data



    # iterate through each batch of the eval data
    for i, batch in enumerate(tqdmn(eval_data)):



      # do not calculate the gradients
      with torch.no_grad():
          # move the batch tensors to the same device as the model
          batch = { k: v.to(device) for k, v in batch.items() }
          # send 'input_ids', 'attention_mask' and 'labels' to the model
          outputs = model(**batch)
          loss = outputs[0]
          #print("loss",loss)


      # get the sentence lengths
      s_lengths = batch['attention_mask'].sum(dim=1)
      # iterate through the examples
      for idx, length in enumerate(s_lengths):
          # get the true values
          true_values = batch['labels'][idx][:length]
          true_labels.extend(true_values.tolist())
          #print("true_values",len(true_values))
          # get the predicted values
          pred_values = torch.argmax(outputs[1], dim=2)[idx][:length]

          pred_values = pred_values.tolist()
          # pred_values = [val for val in pred_values]
          # if len(pred_values) != len(true_values):
          #   print("pred_values_____________________________",len(pred_values))
          predicted.append(pred_values)
          predicted_for_classification_report.extend(pred_values)

          # go through all true and predicted values and store them in the confusion matrix
          # for true, pred in zip(true_values, pred_values):
          #     confusion[true.item()][pred.item()] += 1
    return predicted, predicted_for_classification_report





In [ ]:
valid_predicted, valid_predicted_for_classification_report= prediction(best_model_path, valid_data)

In [ ]:
test_predicted, valid_predicted_for_classification_report= prediction(best_model_path, test_data)

In [ ]:
def convert_id2label_list(list_of_id):
  converted=[]
  for i in list_of_id:
    converted.append(id2label[i])
  return converted

In [ ]:
id2label

# Post processing ( shrinking of expanded labels) and score calculation

In [ ]:
#label post allignment
def generate_score(word_ids_of_sentences,predicted,shrink_labels,true_flat_labels):
#     print(word_ids_of_sentences[0])
#     print(len(shrink_labels[0]))
#     print(shrink_labels[0])

#     print(predicted[0])

    #all_tags = predicted

    shrink_predicted = []

    for i in range(len(word_ids_of_sentences)):
        #print(i)

        predicted[i] = predicted[i][1:-1]

        tags = []
        for j in range(len(word_ids_of_sentences[i])):

            if(j == 0 or word_ids_of_sentences[i][j] == word_ids_of_sentences[i][j-1]+1):
                tags.append(predicted[i][j])


        shrink_predicted.append(tags)

    #predicted_for_classification_report = [item for sublist in new_all_tags for item in sublist]
    if len(shrink_predicted) != len(shrink_labels):
        print("generated tags samples are not eqal number of sampales" )
    predicted_for_classification_report =[]
    allign_predicted_for_sentence=[]

    not_equal = 0
    for i in  range(len(shrink_labels)):
        if len(shrink_labels[i]) == len(shrink_predicted[i]):
            allign_predicted_for_sentence.append(shrink_predicted[i])
            predicted_for_classification_report = predicted_for_classification_report+shrink_predicted[i]

        elif len(shrink_labels[i]) > len(shrink_predicted[i]):
            not_equal+=1
            allign_predicted_for_sentence.append(shrink_predicted[i] +[0] * (len(shrink_labels[i]) - len(shrink_predicted[i])))
            predicted_for_classification_report = predicted_for_classification_report+shrink_predicted[i] +[0] * (len(shrink_labels[i]) - len(shrink_predicted[i]))

        else :
            not_equal+=1
            allign_predicted_for_sentence.append(shrink_predicted[i][:len(shrink_labels[i])])
            predicted_for_classification_report = predicted_for_classification_report+shrink_predicted[i][:len(shrink_labels[i])]


    if len(predicted_for_classification_report) != len(true_flat_labels):
        print("generated tags count are not equal number of count" )


    print("not_equal =", not_equal)



    ######## classification_report


    target_names = labels_list
    print(classification_report(true_flat_labels, predicted_for_classification_report, target_names=target_names,labels=[i for i in range(num_labels)]))

    target_names_1 =labels_list_1

    true_labels = [convert_id2label_list(id_list) for id_list in  shrink_labels]
    predicted_labels = [convert_id2label_list(id_list) for id_list in shrink_predicted]

    evaluator = Evaluator(true_labels,predicted_labels, tags=target_names_1, loader="list")

    results, results_by_tag = evaluator.evaluate()
    macro_f1=0
    for tag in results_by_tag.keys():
        print(tag +" F1 score = ", results_by_tag[tag]["strict"]["f1"])
        macro_f1=macro_f1 + results_by_tag[tag]["strict"]["f1"]
    print("Avrrage macro f1 score = ",macro_f1/6)
    #print(results)

    print("Strict f1 score = ",results["strict"]["f1"])
    return predicted_labels


In [ ]:
len(labels_list_1)

# Score at augmented  valid data

In [ ]:
valid_predicted_labels=generate_score( word_ids_of_valid_sentences,copy.deepcopy(valid_predicted),valid_labels,valid_true_flat_labels)

# Score at assignment  valid data

# Prediction on test data. Ignore score ( all true value of labels of test data are set to 'O' for ease of implimentation.)

In [ ]:
test_predicted_labels=generate_score( word_ids_of_test_sentences,copy.deepcopy(test_predicted),test_labels,test_true_flat_labels)

#  Generation of text file for test data prediction. Each word will be in a new # # line along with its labels separated
# by a tab
# There will be a line gap between sentences.

In [ ]:
# Generation of text file for test data prediction. Each word will be in a new line along with its labels separated
# by a tab
#There will be a line gap between sentences.

#As this work fixed tokenizer max length to 92 some sentences, which length after tokenization is greater than 92
#are cut down. Therefore some dummy labels ('O') appended to there prediction for writing the test data prediction
#file
file = open(best_model_path+"/"+lan+".pred.conll","w")

print(best_model_path+"/"+lan+".pred.conll")
out_of_max_len=0
for i in range(len(test_pdf["sentences"])):

    if len(test_predicted_labels[i]) != len(test_pdf["sentences"][i]):
        print(test_predicted_labels[i])
        print(test_pdf["sentences"][i])

        out_of_max_len+=1
    dummy_labels = ['O'] * (len(test_pdf["sentences"][i]) - len(test_predicted_labels[i]))
    allign_prediction = test_predicted_labels[i] + dummy_labels
    for j in range(len(test_pdf["sentences"][i])):
        #print(allign_prediction[j])
        file.write(allign_prediction[j] + "\n")

    file.write("\n")
file.close()
print("test_data_prediction .txt saved")
print(out_of_max_len," sentences has some dumy label appended for filling the gap labels")

In [ ]:
best_model_path

In [ ]:
len(test_prediction)

In [ ]:
a='''


'''
for i in a :
    print(ord(i))

In [ ]:
model

In [ ]:
x = torch.randn(2, 3)


In [ ]:
y = torch.randn( 2,3)
y

In [ ]:
torch.cat((x, y), 1)

In [ ]:
torch.cat((x, y), 0)

In [ ]:
torch.cat((x, x, x), 0)


In [ ]:
torch.cat((x, x, x), 1)

In [ ]:
import torch

# Create tensors
tensor1 = torch.tensor([[1, 2, 3],
                        [4, 5, 6]])

tensor2 = torch.tensor([10, 20, 30])

# Broadcast tensor2 to the shape of tensor1
broadcasted_tensor2 = tensor2.unsqueeze(0)  # Add a dimension at the beginning
print("Broadcasted tensor2 shape:", broadcasted_tensor2.shape)

# Perform element-wise addition
result = tensor1 + broadcasted_tensor2
print("Result after broadcasting and addition:")
print(result)


In [ ]:
x = torch.tensor([1, 2, 3])



In [ ]:
x.repeat(4,)


In [ ]:
x.repeat(4, 2, 1).size()

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image

# Load the image
image = Image.open("/home/naive123/nlp/Sumit/mner/downloaded_images/20001.jpg")

# Define a transformation to convert the image to a tensor and normalize it
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])

# Apply the transformation to the image
input_tensor = transform(image).unsqueeze(0)  # Add a batch dimension

# Define a feedforward layer to flatten the image
flatten_layer = nn.Flatten()

# Flatten the image into a 1D tensor
output_tensor = flatten_layer(input_tensor)

# Convert the output tensor to a 1D array
output_array = output_tensor.squeeze().numpy()

print("Shape of the flattened array:", output_array.shape)
print("Flattened array:", output_array)


In [ ]:
url = '/home/naive123/nlp/Sumit/mner/downloaded_images/20002.jpg'
image = Image.open(url)

processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
inputs = processor(images=image, return_tensors="pt")

outputs = model(**inputs)
last_hidden_states = outputs.last_hidden_state

In [ ]:
last_hidden_states[0][0].shape

In [ ]:
!pip install timm

